## 라이브러리 import 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from tensorflow import keras

In [ ]:
# 재현성을 위한 코드
random_seed = 777

tf.keras.utils.set_random_seed(random_seed)
# tf.random.normal([5], 0, 1)

## 데이터 불러오기

In [ ]:
# 레이블 불러오기
TrainLabel = np.array(pd.read_csv('./ProcessedData/TrainLabel.csv', sep = ",", header = None))
ValidLabel = np.array(pd.read_csv('./ProcessedData/ValidLabel.csv', sep = ",", header = None))

TrainLabel.shape, ValidLabel.shape

In [ ]:
# 데이터 갯수 정의

NoOfData  = 180                    # 데이터 개수
NoOfValid = int(NoOfData/6)        # 검증 데이터 개수
NoOftrain = NoOfData - NoOfValid   # 학습 데이터 개수

NoOfData, NoOfValid, NoOftrain

In [ ]:
# 데이터 길이 정의
DataLength = 2774 # 데이터 길이

DataLength

In [ ]:
# 전체 가속도 데이터 만들기

NoOfData   = 180      # 데이터 개수 (정상/고장 각각)
TotalData = np.zeros([NoOfData*2, DataLength]) # 데이터 빈집

for i in range(NoOfData):
    tempNormalPath   = './SpotWeldingData/Normal_%d.csv'%(i+1)
    tempAbnormalPath = './SpotWeldingData/Abnormal_%d.csv'%(i+1)
    tempNormal   = pd.read_csv(tempNormalPath  ).iloc[:,0]  # 가속도 데이터
    tempAbnormal = pd.read_csv(tempAbnormalPath).iloc[:,0]  # 가속도 데이터
    TotalData[i,:]            = tempNormal
    TotalData[i + NoOfData,:] = tempAbnormal

TotalData.shape

## STFT 실행 (Spectrogram 생성)

In [ ]:
NoOfSensor = 1
Fs         = 12800

In [ ]:
# STFT test
f,t,Sxx = signal.spectrogram(TotalData, Fs, nperseg = 99, noverlap = 45)
print("STFT Result            :", Sxx.shape)

In [ ]:
plt.figure(figsize=(9,3))
plt.plot(np.arange(0, 0.2167, 1/12800), TotalData[0,:])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (g)')
plt.title('Spot welding acceleration')

plt.show()

In [ ]:
plt.figure(figsize=(9,6))
plt.pcolormesh(t, f, Sxx[0], cmap = 'hot', alpha = 0.7)

plt.xlabel('Time(s)', fontsize=12)
plt.ylabel('Frequency(Hz)', fontsize=12)
plt.title('STFT Result Case 1', fontsize=20)
plt.colorbar()
plt.show()

In [ ]:
# Data Reshape for CNN
TotalData_STFT = Sxx.reshape(Sxx.shape[0], Sxx.shape[1], Sxx.shape[2], NoOfSensor)
print("Reshape_for_CNN Result :", TotalData_STFT.shape)

## 학습 데이터와 검증 데이터 분할

In [ ]:
# 정상/고장 데이터 분할

Normal   = TotalData_STFT[:NoOfData, :, :, :]
Abnormal = TotalData_STFT[NoOfData:, :, :, :]

Normal.shape, Abnormal.shape

In [ ]:
# 정상 및 고장 별 학습/검증 데이터 분할

Normal_train   = Normal[  :NoOftrain, :, :, :]
Abnormal_train = Abnormal[:NoOftrain, :, :, :]

Normal_valid   = Normal[  NoOftrain:, :, :, :]
Abnormal_valid = Abnormal[NoOftrain:, :, :, :]

Normal_train.shape, Abnormal_train.shape, Normal_valid.shape, Abnormal_valid.shape

In [ ]:
# 학습 및 검증 데이터 분할

TrainData = np.concatenate([Normal_train, Abnormal_train], axis = 0)
ValidData = np.concatenate([Normal_valid, Abnormal_valid], axis = 0)

TrainData.shape, ValidData.shape

## CNN(Convolution Neural Network) hyperparameter 설정

In [ ]:
learningRate  = 0.001
Epoch         = 500

## Keras 기반 CNN 구조(Architecture) 설계

In [ ]:
def CNN(input_data):
    keras.backend.clear_session()
    tf.keras.utils.set_random_seed(random_seed)

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(shape=(input_data.shape[1],input_data.shape[2],input_data.shape[3])))             # Input layer
    model.add(keras.layers.Conv2D(filters = 4, kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu))  # Convolution layer 1
    model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)))                                              # Pooling layer 1
    model.add(keras.layers.Conv2D(filters = 8, kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu))   # Convolution layer 2
    
    # Convolution/Pooling layer to Output layer
    model.add(keras.layers.GlobalAveragePooling2D())                                                                    # Global Average Pooling (Simple Structure)

    model.add(keras.layers.Dense(units = 2, activation=keras.activations.softmax))                                      # Output Layer

    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    return model

## CNN(Convolution Neural Network) 학습 및 평가 방법

In [ ]:
# 모델 만들기
CNN_model = CNN(TrainData)

# 모델 구조, 파라미터 개수 확인
CNN_model.summary()

## Keras 기반 CNN 학습 및 평가

In [ ]:
# 모델 학습/검증
hist = CNN_model.fit(TrainData, TrainLabel, epochs=Epoch, verbose = 1)
Loss, Accuracy = CNN_model.evaluate(TrainData,  TrainLabel, verbose=0)

print('Final Loss and Accuracy: {:.4f}, {:.2f}%'.format(Loss, Accuracy*100))

In [ ]:
# 학습 과정 확인(Loss, Accuracy)
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], label='train loss', color = "tab:red")
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['accuracy'], label='train acc', color = "tab:blue")
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
Loss, Accuracy = CNN_model.evaluate(ValidData,  ValidLabel, verbose=0)

print('Validation Accuracy: {:.2f}%'.format(Accuracy*100))

In [ ]:
# 모델 저장
CNN_model.save('./MLmodels/CNN_model.keras')

## CNN model 불러와서 진단

In [ ]:
# 모델 불러오기
LoadedModel = keras.models.load_model('./MLmodels/CNN_model.keras')

In [ ]:
Loss, Accuracy = LoadedModel.evaluate(ValidData,  ValidLabel, verbose=0)

print('[Performance of CNN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

## ++. Confusion Matrix 확인

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
# 분류 결과 확인
Pred = LoadedModel.predict(ValidData)
pd.DataFrame(Pred)

In [ ]:
# One-hot encoding -> Label encoding
true = np.argmax(ValidLabel, axis=-1)
pred = np.argmax(Pred, axis=-1)

# Confusion Matrix 생성
CM = confusion_matrix(true, pred).T

# 데이터의 실제/분류결과 이름
x = ['Normal_True', 'Abnormal_True']
y = ['Normal_Pred', 'Abnormal_Pred']

CM_df = pd.DataFrame(CM,columns=x, index=y)
CM_df

In [ ]:
# heatmap font size
sns.set(font_scale=1.2)

# Heatmap 형태로 확인
fig = plt.subplots(figsize=(8,8))


# 퍼센트
sns.heatmap(CM_df/np.sum(CM_df),
            cmap='Blues',          # cmap Color
            annot=True,            # Value Text
            fmt=".1%",             # Value type (integer = "d")
            linewidths=2)

# 갯수
# sns.heatmap(CM_df,
#             cmap='Blues',        # cmap Color
#             annot=True,          # Value Text
#             fmt="d",             # Value type (integer = "d")
#             linewidths=2) 



plt.show()